In [1]:
import os

import gdal
from plio.io.io_gdal import GeoDataset

# Paths, Files and Variables

Each cube requires a "basepath" and the cubes name.

The basepath is the path to the directory containing the cubes. Ideally, you would want to place mutiple cubes within the same directory
to make opening and accessing them easier. In the cell bellow, define various basepaths and cub names.

You then can define the two cubes however you want using the basepath and a cube.

This also allows you to define the resampleing algorithm to use in the gdal warp and the resolution size of the tiff after the warp.

In [5]:
hirise_basepath = '/home/acpaquette/repos/ortho_map_comparison_code/TestData/HiRISE_Jezero/'
ctx_20_basepath = '/home/acpaquette/repos/ortho_map_comparison_code/TestData/CTX_Jezero/20m_ctx'
ctx_6_basepath = '/home/acpaquette/repos/ortho_map_comparison_code/TestData/CTX_Jezero/6m_ctx'
hrsc_basepath = '/home/acpaquette/repos/ortho_map_comparison_code/TestData/HRSC_Jezero/'

hirise_dem1 = os.path.join(hirise_basepath, 'DEM_1m_Jezero_CE_isis3.cub')
hirise_dem2 = os.path.join(hirise_basepath, 'DEM_1m_Jezero_C_isis3.cub')

ctx_dem1 = os.path.join(ctx_6_basepath, 'tfm_abso_Jezero_F05_V6_IAUsph_adj_XYZposAndVelAndAngles_20m_onePassAfterngate.cub')
ctx_dem2 = os.path.join(ctx_6_basepath, 'tfm_abso_Jezero_J03_V6_IAUsph_adj_XYZposAndVelAndAngles_20m_onePassAfterngate.cub')

# setup the paths to cubes and tiffs
hirise_cub1 = "ESP_023524_1985_1m_o_isis3.cub"
hirise_cub2 = "ESP_048908_1985_1m_o_isis3.cub"
ctx_cub1 = "F05_037607_2008_XN_20N282W_v6_PosAndVelAndAngles_20m_o.cub"
ctx_cub2 = "J03_045994_1986_XN_18N282W_v6_20m_o.cub"
ctx_cub3 = "F05_037607_2008_XN_20N282W_v6_PosAndVelAndAngles_6m_o.cub"
ctx_cub4 = "J03_045994_1986_XN_18N282W_v6_6m_o.cub"
hrsc_cub = "H5270_0000_ND4.IMG"

# cub_image1 = os.path.join(hirise_basepath, hirise_cub1)
# cub_image2 = os.path.join(hirise_basepath, hirise_cub2)

# cub_image1 = os.path.join(ctx_6_basepath, ctx_cub3)
# cub_image2 = os.path.join(ctx_6_basepath, ctx_cub4)

cub_image1 = os.path.join(ctx_20_basepath, ctx_cub1)
cub_image2 = os.path.join(ctx_20_basepath, ctx_cub2)

# cub_image1 = hirise_dem1
# cub_image2 = hirise_dem2

# cub_image1 = ctx_dem1
# cub_image2 = ctx_dem2

# Resampling Algorithm
resample_type = 'bilinear'

# Resolution (x meters per pixel)
res = 20

### Function definitions for applying the image process

In [6]:
def generate_worldfile(file):
    
    world_file = os.path.splitext(file)[0] + ".tfw"
    
    with open(world_file, "w") as fp:
        
        cub_geo = GeoDataset(file)
        geotransform = cub_geo.geotransform
        
        if geotransform is not None:
            x, x_size, x_rot, y, y_rot, y_size = geotransform
            fp.write('%s\n' % x_size)
            fp.write('%s\n' % x_rot)
            fp.write('%s\n' % y_rot)
            fp.write('%s\n' % y_size)
            fp.write('%s\n' % x)
            fp.write('%s\n' % y)
            fp.close()

def translate_image(in_image, out_image, overlap_points, res, resample_type, in_srs, out_srs, out_type = "GTiff"):
    ul, lr = overlap_points

    minx, miny= ul
    maxx, maxy = lr

    fp = gdal.Translate(out_image, in_image, srcWin = [minx, miny, maxx - minx, maxy - miny])
    del(fp)

    fp = gdal.Warp(out_image, out_image, targetAlignedPixels=True, xRes = res, yRes = res, resampleAlg = resample_type, srcSRS = in_srs, dstSRS = out_srs, format = out_type)
    del(fp)
    
def process_coreg_cubs(cub_image1, cub_image2, res, resample_type, out_type = 'GTiff', gen_worldfile = False):
    
    if gen_worldfile:
        generate_worldfile(cub_image1)
        generate_worldfile(cub_image2)
    
    tiff_image1 = os.path.splitext(cub_image1)[0]
    tiff_image2 = os.path.splitext(cub_image2)[0]

    src_in_srs = gdal.Info(cub_image1, format='json')['coordinateSystem']['wkt']
    dst_in_srs = gdal.Info(cub_image2, format='json')['coordinateSystem']['wkt']

    if 'Equirectangular' in src_in_srs:
        src_out_srs = src_in_srs
    else:
        if 'Equirectangular' in dst_in_srs:
            src_out_srs = dst_in_srs

    if 'Equirectangular' in dst_in_srs:
        dst_out_srs = dst_in_srs
    else:
        if 'Equirectangular' in src_in_srs:

            dst_out_srs = src_in_srs
            
    # Get all geo data for the two co-registered tiffs
    cub_geo1 = GeoDataset(cub_image1)
    cub_geo2 = GeoDataset(cub_image2)

    overlap = cub_geo1.compute_overlap(cub_geo2)[0]

    overlap_points1 = [cub_geo1.latlon_to_pixel(i[1], i[0]) for i in overlap][::2]
    overlap_points2 = [cub_geo2.latlon_to_pixel(i[1], i[0]) for i in overlap][::2]
    
    translate_image(cub_image1, tiff_image1, overlap_points1, res, resample_type, src_in_srs, src_out_srs, out_type=out_type)
    translate_image(cub_image2, tiff_image2, overlap_points2, res, resample_type, dst_in_srs, dst_out_srs, out_type=out_type)

### Start image processing

In [7]:
process_coreg_cubs(cub_image1, cub_image2, res, resample_type, out_type='GTiff', gen_worldfile=False)